In [ ]:
%%capture
!pip install langchain-openai
!pip install langchain
!pip install langchain_community

##Reading the data
Using all the functions from previous script to read the data.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, json
import numpy as np
from argparse import ArgumentParser
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
from pprint import pprint
from datetime import datetime
import copy

In [ ]:
#@title Reading train, dev and test data (use own folder path here and chec file names)
folder_path = 'your folder-path'

def read_json_file(name):
    with open(name, 'r') as f:
        data = json.load(f)
        return data

def read_data():
    train = read_json_file(os.path.join(folder_path, "train.json"))
    dev = read_json_file(os.path.join(folder_path, "dev.json"))
    test = read_json_file(os.path.join(folder_path, "test.json"))

    return train, dev, test

train, dev, test = read_data()
print(len(train), len(dev), len(test))

246 50 100


In [ ]:
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain import HuggingFaceHub
import time
from langchain_community.llms import HuggingFaceEndpoint
from google.colab import files

In [ ]:
pro_token = "use your token"
os.environ["HUGGINGFACEHUB_API_TOKEN"]=pro_token

##GPT-4o models

### zero-shot batch extraction.
with both batch and isolated settings.

In [ ]:
# import langchain
# langchain.debug = False

In [ ]:
#create the chain for LLM to invoke
def GPT_batch_extraction_prompt_chain(model):
    prompt_template = PromptTemplate(
        input_variables=['instruction', 'role_descriptions', 'post', 'comment', 'type'],
        template = '''
        ##Instruction##
        {instruction}

        ##Arguments {type}##
        {role_descriptions}

        ##Post##
        {post}

        ##Comment##
        {comment}

        ##JSON##
        '''
    )

    prompt_chain = prompt_template | model | StrOutputParser()
    return prompt_chain


In [ ]:
#If one argument has multiple values separate them by commas.
#returning the outputs by invoking the chain

def getting_GPT_batch_outputs(model, data, instruction, role_details, extraction_type):
    arg_types = ['main-arguments', 'event-specific-arguments', 'subject-effect-arguments']
    predictions = []
    invoke_count = 0
    for dt in data:
          data_sample = dt
          post, comment, label = dt['text1'], dt['text2'], dt['label']
          print(dt['doc_id'])
          raw_predictions = {}
          for arg_typ in arg_types:
              role_descriptions = role_details[label][arg_typ] #chosing the argument details from the correspond event
              roles = list(role_descriptions.keys())
              input_dict = {
                'instruction' : instruction,
                'post' : post,
                'comment' : comment,
                'role_descriptions' : role_descriptions,
                'type': extraction_type
              }
              prompt_chain = GPT_batch_extraction_prompt_chain(model) ##creating the prompt chain
              while True: ##to get rid of model overload error
                  try:
                      output = prompt_chain.invoke(input_dict) #invoking the LLM to get prediction
                      break
                  except Exception as e:
                      print(e)
                      time.sleep(3)
              invoke_count+=1 #Checking how many times I am doing the inference.
              raw_predictions[arg_typ] = output

              # print(label, arg_typ)
              # pprint(raw_predictions[arg_typ])
          data_sample['raw-predictions'] = raw_predictions ##saving the raw-prediction will refine them later
          predictions.append(data_sample)
    print(model, extraction_type, invoke_count)
    return predictions

In [ ]:
%%time
def read_json_file(name):
    folder_path = 'path'
    name = os.path.join(folder_path, name)
    with open(name, 'r') as f:
        data = json.load(f)
        return data

def save_json(json_data, file_name):
    json_data = json.dumps(json_data)
    print(file_name)
    with open(file_name, "w") as json_file:
          json_file.write(json_data)

def get_predictions(model, new_data, nm_model):
      xx = 0
      prompt_types = ['batch']
      exp_types = ['description_guided', 'question_guided']

      predictions = {}
      openai_api_key = "your-key" # this is the lab key - make sure to delete or hide eventually
      llm_oi = ChatOpenAI(openai_api_key=openai_api_key, temperature=0.0, model='gpt-4o-2024-05-13') #this is the chat model.

      # llm_hf = HuggingFaceEndpoint(repo_id= model, temperature=0.0, max_new_tokens=128)

      for p_type in prompt_types:
          for e_type in exp_types:
              ## getting the predefined instruction template and event_roles
              instruction_template = read_json_file('instruction_template.json')
              event_roles = read_json_file(f'role_definitions_{e_type}.json')
              #instruction = instruction_template[f'{p_type}-{e_type}']
              instruction = '''Extract the following arguments from the post and comment. Do not use more than 12 words to describe an agrument. Give arguments in JSON format and put 'null' if value not specified.'''
              data_to_predict = new_data #reinitilizing the data again again

              print(model, p_type, e_type)
              # type of experiment question-guided or description guided
              if e_type=='description_guided':
                  preds = getting_GPT_batch_outputs(llm_oi, data_to_predict, instruction, event_roles, 'Descriptions')
              else:
                  preds = getting_GPT_batch_outputs(llm_oi, data_to_predict, instruction, event_roles, 'Questions')

              current_time = datetime.now().replace(second=0, microsecond=0).time()
              current_date = datetime.now().replace(second=0, microsecond=0).date()

              nm = f'preds_{nm_model}_{p_type}_{e_type}_{current_date}_{current_time}.json'
              save_json(preds, nm)

              print("### ", p_type, e_type, "###")
              #this is very very important
              predictions[f'{p_type}_{e_type}'] = copy.deepcopy(preds) #saving the outputs

      return predictions ## return the predictions for a model.


CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.11 µs


In [ ]:
%%time
model_name = ['gpt-4o'] ## for saving the models in the folder
lm_models = ['gpt-4o-2024-05-13']

i = 0
for x in range(1, 3):
    for model in lm_models:
        with get_openai_callback() as cb:
            predictions = get_predictions(model, test,  model_name[i]) ## right now I am doing predictions on dev set.
            nm = f'{x}-{model_name[i]}_test_100_predictions.json'
            save_json(predictions, nm)
            folder_path = 'save-path'
            save_json(predictions, os.path.join(folder_path, nm))

        print(cb)